<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_08_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/California_housing.csv'
data = pd.read_csv(url)

base = data.dropna()

In [2]:
# [1] corr

corr = base[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'AveOccup',
             'Target']].corr(method='pearson')

corr = corr.assign(abs_corr=corr.Target.apply(lambda x: abs(x)))
corr = corr.sort_values(by='abs_corr', ascending=False)

top2 = ', '.join(list(corr.abs_corr.index[1:3]))

print(f'Answer [1] : {top2}')

Answer [1] : MedInc, AveRooms


In [3]:
# [2] LinearRegression

from sklearn.linear_model import LinearRegression
import math

# set X variables
X_cols = list(corr.abs_corr.index[1:-1]) + ['Population']

# prepare train set
train_X = base[base.Xgrp == 0][X_cols]
train_y = base[base.Xgrp == 0].Target

# train
lr = LinearRegression()
model = lr.fit(train_X, train_y)

# predict
sample = pd.DataFrame([[4, 4, 37, 1, 1200]], columns=X_cols)
pred2 = model.predict(sample)

print(f'Answer [2] : {math.floor(pred2[0] * 1000) / 1000}')

Answer [2] : 2.485


In [4]:
# [3] root mean_squared_error

from sklearn.metrics import mean_squared_error
import numpy as np
import math

# prepare validation set
val_X = base[base.Xgrp == 1][X_cols]
val_y = base[base.Xgrp == 1].Target

# predict
pred3 = model.predict(val_X)

# calculate room mean squared error
rmse = np.sqrt(mean_squared_error(val_y, pred3))

print(f'Answer [3] : {math.floor(rmse * 1000) / 1000}')

Answer [3] : 0.794


In [5]:
# [4] StandardScaler, PCA

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math

# normalize and PCA
norm_data = StandardScaler().fit_transform(base[base.columns[0:-2]])
pca = PCA(n_components=6).fit(norm_data)

# find eigen values
print(f'Answer [4] : {math.floor(pca.explained_variance_[2] * 1000) / 1000}')

Answer [4] : 1.023


In [17]:
# [5] list comprehension, enumerate, LinearRegression - R-Squared(Score)

from sklearn.linear_model import LinearRegression

# find top 5 principal component index
ind = [i for i, v in enumerate(pca.explained_variance_)
         if v != pca.explained_variance_.min()]

# prepare test set
test_X = pca.transform(norm_data)
test_X = pd.DataFrame(test_X)
test_y = base.Target

# select top 5 component
test_X = test_X[ind]

# train
lr = LinearRegression()
model = lr.fit(test_X, test_y)

# predict
pred5 = model.predict(test_X)

# find score
r_squared = model.score(test_X, test_y)

print(f'Answer [5] : {math.floor(r_squared * 1000) / 1000}')

Answer [5] : 0.492
